In [ ]:
!pip install pyngrok==4.1.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15979 sha256=5119f9b6cff8942d05b48e9265db926bf0e67aeb3f3bf6a24149ec18ca096d50
  Stored in directory: /root/.cache/pip/wheels/89/2d/c2/abe6bcfde6bce368c00ecd73310c11edb672c3eda09a090cfa
Successfully built pyngrok


In [ ]:
!pip install --upgrade streamlit
import streamlit as st

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 63.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.0 MB/s eta 0:00:00
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19579 sha256=cb316cb76218045e0127b8e950f628d94a78f3d99d5975aff6cefb07aaa144ea
  Stored in directory: /root/.cache/pip/wheels/2d/f0/a8/1094fca7a7e5d0d12ff56e0c64675d72aa5cc81a5fc200e849
Successfully built validators


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [ ]:
cd /content/gdrive/My Drive/DS440/Data/image

/content/gdrive/.shortcut-targets-by-id/1HuqlSiQQRulpqyyP747KMhMiCeBTkfqs/DS440/Data/image


In [ ]:
!ls

12345.jpg  best_model.h5  features.pkl	model.png
app.py	   captions.txt   Images	tokenizer.pkl


In [ ]:
pip install translate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from translate import Translator

def translate_text(text, target_language = "ja"):
  translator = Translator(to_lang=target_language)
  translation = translator.translate(text)
  return translation

In [ ]:
from translate import Translator

def translate_text(text, target_language):
    translator = Translator(to_lang=target_language)
    translation = translator.translate(text)
    return translation

In [ ]:
print(translate_text("how are you", "ja"))

お元気ですか


In [ ]:
%%writefile app.py
import os   # handling the files
import pickle # storing numpy features
import numpy as np
from tqdm.notebook import tqdm # how much data is process till now
from keras.models import load_model
import cv2

from tensorflow.keras.applications.vgg16 import VGG16 , preprocess_input # extract features from image data.
from tensorflow.keras.preprocessing.image import load_img , img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.layers import Input , Dense , LSTM , Embedding , Dropout , add

import streamlit as st
import pickle
import numpy as np
import pandas as pd
import string
from translate import Translator

Navigation = {"page_title":"ImageCaptionGenerator.io","page_icon":"👽","layout":"centered"}
st.set_page_config(**Navigation)


def translate_text(text, target_language):
    translator = Translator(to_lang=target_language)
    translation = translator.translate(text)
    return translation


def idx_to_word(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None

# generate caption for an image
def predict_caption(model, image, tokenizer, max_length):
    # add start tag for generation process
    in_text = 'startseq'
    # iterate over the max length of sequence
    for i in range(max_length):
        # encode input sequence
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        # pad the sequence
        sequence = pad_sequences([sequence], max_length)
        # predict next word
        yhat = model.predict([image, sequence], verbose=0)
        # get index with high probability
        yhat = np.argmax(yhat)
        # convert index to word
        word = idx_to_word(yhat, tokenizer)
        # stop if word not found
        if word is None:
            break
        # append word as input for generating next word
        in_text += " " + word
        # stop if we reach end tag
        if word == 'endseq':
            break
    return in_text

from PIL import Image
import matplotlib.pyplot as plt
# def generate_caption(image_name):
#     # load the image
#     # image_name = "1001773457_577c3a7d70.jpg"
#     image_id = "1"

#     # predict the caption
#     y_pred = predict_caption(model, features[image_id], tokenizer, max_length)
#     # print('--------------------Predicted--------------------')
#     # print(y_pred)
#     # plt.imshow(image)



def load_tokenizer():
    with open('tokenizer.pkl', 'rb') as handle:
        tokenizer = pickle.load(handle)
    return tokenizer

def load_image(img):
    im = Image.open(img)
    image = np.array(im)
    return image

  
tokenizer = load_tokenizer()
model = load_model("/content/gdrive/My Drive/DS440/Data/image/best_model.h5")
max_length = 35

def main():
    st.header("_ImageCaption_:blue[Generator]")

    st.write("""### Please upload your image:""")


    # construct text box
    input_buffer = st.file_uploader("Upload an image", type=("png", "jpg", "jpeg"))
    option = st.selectbox("language", ("en", "ja", "ko", "pt", "es", "zh", "zh-TW"))
    button = st.button("Generate")

    if button:
      if input_buffer:
        # extract features from image
        features = {}
        image_name = input_buffer
        # Load vgg16 Model
        model_vgg = VGG16()
        # restructure model
        model_vgg = Model(inputs = model_vgg.inputs , outputs = model_vgg.layers[-2].output)
        # load the image from file
        image = load_img(image_name, target_size=(224, 224))
        # convert image pixels to numpy array
        image = img_to_array(image)
        # reshape data for model
        image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
        # preprocess image for vgg
        image = preprocess_input(image)
        # extract features
        feature = model_vgg.predict(image, verbose=0)
        # get image ID
        image_id = "1"
        # image_name.split('.')[0]
        # store feature
        features[image_id] = feature

        y_pred = (predict_caption(model, features[image_id], tokenizer, max_length))[8:].strip("endseq")
        st.subheader(translate_text(y_pred, option))
        uploaded_img = load_image(image_name)
        st.image(uploaded_img)

      


if __name__ == '__main__':
    main()

Overwriting app.py


In [ ]:
!ngrok

NAME:
   ngrok - tunnel local ports to public URLs and inspect traffic

DESCRIPTION:
    ngrok exposes local networked services behinds NATs and firewalls to the
    public internet over a secure tunnel. Share local websites, build/test
    webhook consumers and self-host personal services.
    Detailed help for each command is available with 'ngrok help <command>'.
    Open http://localhost:4040 for ngrok's web interface to inspect traffic.

EXAMPLES:
    ngrok http 80                    # secure public URL for port 80 web server
    ngrok http -subdomain=baz 8080   # port 8080 available at baz.ngrok.io
    ngrok http foo.dev:80            # tunnel to host:port instead of localhost
    ngrok http https://localhost     # expose a local https server
    ngrok tcp 22                     # tunnel arbitrary TCP traffic to port 22
    ngrok tls -hostname=foo.com 443  # TLS traffic for foo.com to port 443
    ngrok start foo bar baz          # start tunnels from the configuration file

VERSI

In [ ]:
!ngrok authtoken 2GE9pLnERwJ5wZ1vHuGitlRjoD5_327yPZExMjvGdWqRQhQyK

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
from pyngrok import ngrok
public_url = ngrok.connect(port='8501')

INFO:pyngrok.process:ngrok process starting: 883
2023-04-17 22:09:04.037 INFO    pyngrok.process: ngrok process starting: 883
INFO:pyngrok.process:t=2023-04-17T22:09:04+0000 lvl=info msg="no configuration paths supplied"

2023-04-17 22:09:04.056 INFO    pyngrok.process: t=2023-04-17T22:09:04+0000 lvl=info msg="no configuration paths supplied"

INFO:pyngrok.process:t=2023-04-17T22:09:04+0000 lvl=info msg="using configuration at default config path" path=/root/.ngrok2/ngrok.yml

2023-04-17 22:09:04.067 INFO    pyngrok.process: t=2023-04-17T22:09:04+0000 lvl=info msg="using configuration at default config path" path=/root/.ngrok2/ngrok.yml

INFO:pyngrok.process:t=2023-04-17T22:09:04+0000 lvl=info msg="open config file" path=/root/.ngrok2/ngrok.yml err=nil

2023-04-17 22:09:04.074 INFO    pyngrok.process: t=2023-04-17T22:09:04+0000 lvl=info msg="open config file" path=/root/.ngrok2/ngrok.yml err=nil

INFO:pyngrok.process:t=2023-04-17T22:09:04+0000 lvl=info msg="starting web service" obj=we

In [ ]:
!streamlit run app.py & npx localtunnel — port 8501

[#.................] | fetchMetadata: sill resolveWithNewModule follow-redirect


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://35.185.42.222:8501

npx: installed 22 in 6.239s
Usage: lt --port [num] <options>

Options:
  -p, --port                Internal HTTP server port                 [required]
  -h, --host                Upstream server providing forwarding
                                             [default: "https://localtunnel.me"]
  -s, --subdomain           Request this subdomain
  -l, --local-host          Tunnel traffic to this host instead of localhost,
                            override Host header to this host
      --local-https         Tunnel traffic to a local HTTPS server     [boolean]
      --local-cert          Path to certificate PEM file for local HTTPS server
      --local-key           Path to certificate key file for local HTTPS server
      --local-ca            Path to certificate au

INFO:pyngrok.process:t=2023-04-17T22:09:34+0000 lvl=info msg="join connections" obj=join id=b98e7471f9f1 l=127.0.0.1:8501 r=104.39.7.28:65465

2023-04-17 22:09:34.785 INFO    pyngrok.process: t=2023-04-17T22:09:34+0000 lvl=info msg="join connections" obj=join id=b98e7471f9f1 l=127.0.0.1:8501 r=104.39.7.28:65465

INFO:pyngrok.process:t=2023-04-17T22:09:34+0000 lvl=info msg="join connections" obj=join id=9829b7743250 l=127.0.0.1:8501 r=104.39.7.28:65465

2023-04-17 22:09:34.874 INFO    pyngrok.process: t=2023-04-17T22:09:34+0000 lvl=info msg="join connections" obj=join id=9829b7743250 l=127.0.0.1:8501 r=104.39.7.28:65465

INFO:pyngrok.process:t=2023-04-17T22:09:34+0000 lvl=info msg="join connections" obj=join id=9f51ce49fe35 l=127.0.0.1:8501 r=104.39.7.28:65465

2023-04-17 22:09:34.889 INFO    pyngrok.process: t=2023-04-17T22:09:34+0000 lvl=info msg="join connections" obj=join id=9f51ce49fe35 l=127.0.0.1:8501 r=104.39.7.28:65465

INFO:pyngrok.process:t=2023-04-17T22:09:34+0000 lvl=info m

2023-04-17 22:09:36.644815: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-17 22:09:37.764647: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


INFO:pyngrok.process:t=2023-04-17T22:09:44+0000 lvl=info msg="join connections" obj=join id=1be2add12660 l=127.0.0.1:8501 r=104.39.7.28:65465

2023-04-17 22:09:44.317 INFO    pyngrok.process: t=2023-04-17T22:09:44+0000 lvl=info msg="join connections" obj=join id=1be2add12660 l=127.0.0.1:8501 r=104.39.7.28:65465

INFO:pyngrok.process:t=2023-04-17T22:09:44+0000 lvl=info msg="join connections" obj=join id=6cf0be6e16f7 l=127.0.0.1:8501 r=104.39.7.28:65465

2023-04-17 22:09:44.326 INFO    pyngrok.process: t=2023-04-17T22:09:44+0000 lvl=info msg="join connections" obj=join id=6cf0be6e16f7 l=127.0.0.1:8501 r=104.39.7.28:65465

INFO:pyngrok.process:t=2023-04-17T22:09:44+0000 lvl=info msg="join connections" obj=join id=4705afcbf2c6 l=127.0.0.1:8501 r=104.39.7.28:65465

2023-04-17 22:09:44.371 INFO    pyngrok.process: t=2023-04-17T22:09:44+0000 lvl=info msg="join connections" obj=join id=4705afcbf2c6 l=127.0.0.1:8501 r=104.39.7.28:65465

INFO:pyngrok.process:t=2023-04-17T22:09:44+0000 lvl=info m

553467096/553467096 [==============================] - 4s 0us/step


INFO:pyngrok.process:t=2023-04-17T22:12:51+0000 lvl=info msg="received stop request" obj=app stopReq="{err:<nil> restart:false}"

2023-04-17 22:12:51.532 INFO    pyngrok.process: t=2023-04-17T22:12:51+0000 lvl=info msg="received stop request" obj=app stopReq="{err:<nil> restart:false}"

INFO:pyngrok.process:t=2023-04-17T22:12:51+0000 lvl=info msg="session closing" obj=tunnels.session err=nil

2023-04-17 22:12:51.549 INFO    pyngrok.process: t=2023-04-17T22:12:51+0000 lvl=info msg="session closing" obj=tunnels.session err=nil



  Stopping...


In [ ]:
public_url

'http://d96a-35-185-42-222.ngrok-free.app'